In [ ]:
pip install ta


  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.10.2-py3-none-any.whl size=29089 sha256=e84b8286f20f112f7b87dd27ee7cb8b46a5e5becee367e7b4f051654c4dbf4b9
  Stored in directory: /root/.cache/pip/wheels/47/51/06/380dc516ea78621870b93ff65527c251afdfdc5fa9d7f4d248
Successfully built ta


In [ ]:
pip install plotly

In [ ]:
pip install pandas

In [ ]:
import yfinance as yf #getting data
import pandas as pd #data handling
import numpy as np #calc
import ta #tech indi
import matplotlib.pyplot as plt #visualization


In [ ]:
class Backtest:

  def __init__(self,symbol):
    self.symbol=symbol
    self.df=yf.download(self.symbol, start='2019-01-01')

    if self.df.empty:
      print(self.symbol,'no data pulled')
    else:
      # results=pd.DataFrame()
      self.calc_indicators()
      self.generate_signal()
      self.backtest_dataframe()
      self.get_results_dataframe()


      # self.profit=self.calc_profit()
      # # self.max_drawdown= self.profit.min()
      # self.cumul_profit=(self.profit+1).prod()-1
      # # self.plot_chart()

  def calc_indicators(self):
    self.df['ma_20']=self.df.Close.rolling(20).mean()
    self.df['vol']=self.df.Close.rolling(20).std()
    self.df['upper_bb']=self.df.ma_20+(2*self.df.vol)
    self.df['lower_bb']=self.df.ma_20-(2*self.df.vol)
    self.df['rsi']=ta.momentum.rsi(self.df.Close,window=6)
    self.df.dropna(inplace=True)
    self.df.reset_index(inplace=True)

  def generate_signal(self):
    #buy signal
    self.df['signal'] = np.where(
        (self.df['rsi'] < 30) &
        (self.df['Close'] < self.df['lower_bb']), 1, np.nan)

    #sell signal
    self.df['signal'] = np.where(
        (self.df['rsi'] > 70) &
        (self.df['Close'] > self.df['upper_bb']), -1, self.df['signal'])

    #buy/sell next trading day
    self.df['signal'] = self.df['signal'].shift()
    self.df['signal'] = self.df['signal'].fillna(0)



  def plot_chart(self):
    # plt.figure(figsize=(10,5))
    # plt.plot(self.df.Close)

    # plt.scatter(self.buy_arr.index,self.buy_arr.values,marker='^',c='g')
    # plt.scatter(self.sell_arr.index,self.sell_arr.values,marker='v',c='r')

    plt.figure(figsize=(12,5))
# plt.xticks(rotation=45)

    plt.plot(self.df['Date'], self.df['Close'])
    plt.scatter(self.df[(self.df['signal'] == 1)]['buy_date'], self.df[(self.df['signal'] == 1)]['Close'], label = 'Buy', marker='^', c='g')
    plt.scatter(self.df[(self.df['signal'] == -1)]['sell_date'], self.df[(self.df['signal'] == -1)]['Close'], label = 'Sell', marker='v', c='r')
    plt.title('Price Chart & Historical Trades', fontweight="bold")
    return plt
# plt.legend()
# plt.show()

  def backtest_dataframe(self):
    trade_points=pd.DataFrame(columns=["Action","Price","Date"])
    position = 0
    net_profit = 0
    percentage_change = []
    self.df['buy_date'] = ''
    self.df['sell_date'] = ''

    for i in self.df.index:
        close = self.df["Close"][i]
        date = self.df['Date'][i]

        # Buy action
        if self.df["signal"][i] == 1:
            if(position == 0):
                buy_price = close
                position = 1
                self.df.at[i, 'buy_date'] = date
                trade_points=trade_points.append({"Action":"Buy","Price":np.round(buy_price,2),"Date":date},ignore_index=True)
                #print(f"Buying at {str(buy_price)} on {str(date)}")

        # Sell action
        elif (self.df["signal"][i] == -1):
            if(position == 1):
                sell_price = close
                bought = 0
                position = 0
                self.df.at[i, 'sell_date'] = date
                trade_points=trade_points.append({"Action":"Sell","Price":np.round(sell_price,2),"Date":date},ignore_index=True)
                #print(f"Selling at {str(sell_price)} on {str(date)}")

                # Get percentage change of trade
                pc = (sell_price/buy_price-1)*100
                percentage_change.append(pc)
                net_profit += (sell_price - buy_price)

    # Calculate trade statistics
    gains = 0
    ng = 0
    losses = 0
    nl = 0
    totalR = 1

    for i in percentage_change:
        if(i > 0):
            gains += i
            ng += 1
        else:
            losses += i
            nl += 1
        totalR = totalR * ((i/100)+1)

    totalR = round((totalR-1)*100, 2)

    if(ng > 0):
        avgGain = round(gains/ng, 2)
        maxR = round(max(percentage_change), 2)
    else:
        avgGain = 0
        maxR = "undefined"

    if(nl > 0):
        avgLoss = round(losses/nl, 2)
        maxL = round(min(percentage_change), 2)
    else:
        avgLoss = 0
        maxL = "undefined"

    if(ng > 0 or nl > 0):
        win_rate = round((ng/(ng+nl))*100, 2)
    else:
        win_rate = 0

    Metric=['Number of Trades','Number of Gains','Number of Losses','Total Returns %','Win Rate %','Average Gain %','Average Loss %','Max Return %','Max Loss %']
    Value=[ng+nl,ng,nl,totalR,win_rate,avgGain,avgLoss,maxR,maxL]
    results=pd.DataFrame({'Metric': Metric,
        'Value': Value})
    # print()
    # print('Evaluation Metrics:')
    # print('-----------------------------------')
    # print(f"Number of Trades: {ng+nl}")
    # print(f"Number of Gains: {ng}")
    # print(f"Number of Losses: {nl}")
    # print(f"Total Returns: {totalR}%")
    # print(f"Win Rate: {win_rate}%")
    # print(f"Average Gain: {avgGain}%")
    # print(f"Average Loss: {avgLoss}%")
    # print(f"Max Return: {maxR}%")
    # print(f"Max Loss: {maxL}%")
    # print()
    self.results=results
    # print(results)
    return results, trade_points

  def get_results_dataframe(self):
    if hasattr(self, 'results'):
        return self.results
    else:
        return None




In [ ]:
instance=Backtest('AAPL')
print(type(instance.df['buy_date'][0]))


In [ ]:
df=instance.df
print(df[df['signal'] == 1])
#print(df)
x=[]
y=[]
for i in range(0,len(df)):
  if len(str(df['buy_date'][i]))>2:
    #print(df['buy_date'][i])
    x.append(df['buy_date'][i])
    y.append(df['Close'][i])



In [ ]:
# print(b)

In [ ]:

hi=instance.get_results_dataframe()
if hi is not None:
    print(hi)
else:
    print("No results available.")

In [ ]:
a=instance.plot_chart()
type(a)

In [ ]:
pip install dash


In [ ]:
from dash import Dash,Input, Output, dcc, html,callback, dash_table
import io
import base64
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
tickers=pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0].Symbol

In [ ]:
list(tickers)

In [ ]:
app = Dash(__name__)

app.layout = html.Div([
	html.H1('Mean Reversion- Stock Screener'),

    html.Div(
        [
            html.Div([
                dcc.Dropdown(id="stock_name",options=list(tickers),placeholder="Select a Stock"),

                html.Br(),
                html.H4("Strategy Performance",id="heading1",style={"display":"None"}),
                html.Br(),
                html.Div([],id="metrics_table"),

                html.Br(),
                html.H4("Strategy Execution",id="heading2",style={"display":"None"}),
                html.Br(),
                html.Div([],id="execution_table"),
                html.Br(),
                dcc.Graph(id='plot'),

         ] )
        ]

)


           ] )



@callback(
    Output(component_id='heading1', component_property='style'),
    Output(component_id='heading2', component_property='style'),
    Output(component_id='metrics_table', component_property='children'),
    Output(component_id='execution_table', component_property='children'),

    Input(component_id='stock_name', component_property='value')
)
def update_output_div(input_value):
    print(input_value)
    instance=Backtest(input_value)
    a,b=instance.backtest_dataframe()
    #df=backtest_dataframe(df)

    q=dash_table.DataTable(

                           data=a.to_dict('records'),
                           columns=[{"name": i, "id": i} for i in a.columns],
                           style_header={
                                    'backgroundColor': 'rgb(240,248,255)',
                                    'color': 'black',
                                    'fontWeight': 'bold'
                                         },
                           style_cell_conditional=[
                                                {
                                                    'if': {'column_id': c},
                                                    'textAlign': 'center'
                                                } for c in a.columns
                                            ],


                           )
    w=dash_table.DataTable(

                           data=b.to_dict('records'),
                           columns=[{"name": i, "id": i} for i in b.columns],
                           style_header={
                                    'backgroundColor': 'rgb(240,248,255)',
                                    'color': 'black',
                                    'fontWeight': 'bold',
                                    'textAlign': 'center'
                                         },
                           style_data_conditional=[
                                                  {
                                                      'if': {'row_index': 'even'},
                                                      #'backgroundColor': 'rgb(144,238,144)',
                                                      'textAlign': 'center'
                                                  },{
                                                      'if': {'row_index': 'odd'},
                                                      #'backgroundColor': 'rgb(255,114,118)',
                                                      'textAlign': 'center'
                                                  }],




                           )
    return {"display":"block"},{"display":"block"}, q,w

@app.callback(
    Output('plot', 'figure'), # src attribute
    [Input('stock_name', 'value')]
)
def update_figure(input_values):
    instance=Backtest(input_values)
    df=instance.df
    fig = px.line(df, x="Date", y="Close")
    x_data=[]
    y_data=[]
    for i in range(0,len(df)):
      if len(str(df['sell_date'][i]))>2:
        #print(df['buy_date'][i])
        x_data.append(df['sell_date'][i])
        y_data.append(df['Close'][i])

    fig.add_trace(go.Scatter(x=x_data, y=y_data,
                    mode='markers', name='Sell',marker=dict(size=10)))

    x_data=[]
    y_data=[]
    for i in range(0,len(df)):
      if len(str(df['buy_date'][i]))>2:
        #print(df['buy_date'][i])
        x_data.append(df['buy_date'][i])
        y_data.append(df['Close'][i])

    fig.add_trace(go.Scatter(x=x_data, y=y_data,
                    mode='markers', name='Buy',marker=dict(size=10)))



    return fig
if __name__ == '__main__':
	app.run_server(debug=False,host='0.0.0.0',port=8058)
